In [1]:
import pinocchio as pin
import numpy as np
import time
from numpy.linalg import pinv,inv,norm,svd,eig
from utils.tiago_loader import loadTiago
import matplotlib.pylab as plt; plt.ion()
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
from typing import List, Union, Optional, Tuple

Load an extra cube in the viewer to figure a table. First, control the robot hand to reach an arbitrary point on the table (don't worry about collisions). Then, implement a control law to control three tasks:
* The tool frame should be kept on the table = **vertical** constraint  (only the vertical component of the error matters, select the third row of the Jacobian and error accordingly).
* The gaze should reach the position of a ball positionned on the table.
* The center of the mobile base frame should reach a given goal on the floor. For this task, only the horizontal components (x- and y-) of the task matter, select only the first two rows of the Jacobian and error accordingly.

In [2]:
robot = loadTiago(addGazeFrame=True)
viz = MeshcatVisualizer(robot)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [36]:
TABLE_HEIGHT = 0.7

In [50]:
IDX_GAZE = robot.model.getFrameId('framegaze')
IDX_TOOL = robot.model.getFrameId('frametool')
IDX_BASIS = robot.model.getFrameId('framebasis')

# Add a small ball as a visual target to be reached by the robot
# ball = np.array([ 1.2,0.5,1.1 ])
ball = np.array([ 1.2,0.5,TABLE_HEIGHT + 0.2])
o_Mgazegoal = pin.SE3(np.eye(3), ball)
viz.addSphere('ball', .05, [ .8,.1,.5, .8] )
viz.applyConfiguration('ball', list(ball)+[0,0,0,1])

# Add the target on the floor
# basis_target_coordinates = ball + np.array([ 0.5 , 0.2, -ball[-1]])
basis_target_coordinates = ball + np.array([-0.2 , -0.2, -ball[-1]])
o_M_floorTarget = pin.SE3(np.eye(3), basis_target_coordinates)
viz.delete('basis_target')
viz.addBox('basis_target', [.1,.1,.4], [ 1.,0.,0., 1.] )
viz.applyConfiguration('basis_target',o_M_floorTarget)


# Add the table
table_coordinates = np.array([ 1., 1., TABLE_HEIGHT])
# table_coordinates = np.array([ -1.5, 1., TABLE_HEIGHT])
o_Mtable = pin.SE3(np.eye(3), table_coordinates)
viz.delete('table')
viz.addBox('table', [2. , 2.,.01], [ 1., 1., 1., .5] )
viz.applyConfiguration('table',o_Mtable)

# Robot initial configuration.
q0 = np.array([ 0.  ,  0.  ,  1.  ,  0.  ,  0.18,  1.37, -0.24, -0.98,  0.98,
                0.  ,  0.  ,  0.  ,  0.  , -0.13,  0.  ,  0.  ,  0.  ,  0.  ])
viz.display(q0)
pin.framesForwardKinematics(robot.model,robot.data,q0)
# Then get the position
o_Mtool_start = robot.data.oMf[IDX_TOOL]
viz.delete("start")
viz.addBox('start', [.1,.1,.1], [ 1.,.1,.5, .6] )
viz.applyConfiguration('start',o_Mtool_start)


In [38]:
def reset(viz):
    viz.display(q0)
    for i in range(2000): 
        viz.delete(f"debug/traj{i}")
        viz.delete(f"debug/traj_gaze{i}")
    viz.delete("current_6D")
    viz.delete("gaze")
reset(viz)

### Simple testing the position (1/2/3D) constraint
```python
# TOOL STAYS ON THE TABLE
vq = get_config_velocity_update_translation(q, robot, IDX_TOOL, o_Mtable, constraint=(2,3))

# TOOL REACHES THE CENTER OF THE TABLE but not Z
vq = get_config_velocity_update_translation(q, robot, IDX_TOOL, o_Mtable, constraints=(0,2))

# BASIS REACHES 2D XY TARGET
vq = get_config_velocity_update_translation(q, robot, IDX_BASIS, o_M_floorTarget, constraint=(0,2))
```

$ P_1 = I_{nq} - J_1^+ J_1 $

> copyright: someone helped!

$ P_2 = P_1 - {(J_2 P_1)}^+ {J_2 P_1} $

In [72]:
def extract_dim(vec: np.ndarray, start:int, end:int) -> np.ndarray:
    return vec[start:end, ...]

def get_config_velocity_update_translation(
    q: np.ndarray,
    rob: pin.RobotWrapper,
    index_object: int,
    o_Mtarget: pin.SE3,
    constraints: Tuple[int, int] =(0, 3)
) -> Tuple[np.ndarray, np.ndarray]:
    """Make a step toward the constrained position for object
    `index_object`
    """
    o_Mcurrent = rob.data.oMf[index_object]
    obj_Jobj = pin.computeFrameJacobian(rob.model, rob.data, q, index_object, pin.WORLD)
    obj_JobjC = extract_dim(obj_Jobj, *constraints) # constrain over specific dimension
    obj_2_goal = (o_Mtarget.translation - o_Mcurrent.translation)
    obj_2_goalC = extract_dim(obj_2_goal, *constraints) # constrain over specific dimension
    obj_JobjC_inv = np.linalg.pinv(obj_JobjC)
    vq = obj_JobjC_inv @ obj_2_goalC
    projector = np.eye(robot.nv) - obj_JobjC_inv @ obj_JobjC
    return vq, projector


def get_config_velocity_update_translation_with_proj(
    q: np.ndarray,
    rob: pin.RobotWrapper,
    index_object: int,
    o_Mtarget: pin.SE3,
    constraints: Tuple[int, int] =(0, 3),
    projector: np.ndarray=None,
    vq_prev: np.ndarray=None
) -> Tuple[np.ndarray, np.ndarray]:
    
    o_Mcurrent = rob.data.oMf[index_object]
    obj_Jobj = pin.computeFrameJacobian(rob.model, rob.data, q, index_object, pin.WORLD)
    obj_JobjC = extract_dim(obj_Jobj, *constraints) # constrain over specific dimension
    obj_2_goal = (o_Mtarget.translation - o_Mcurrent.translation) # v2*
    obj_2_goalC = extract_dim(obj_2_goal, *constraints) # constrain over specific dimension
    
    new_error = (obj_2_goalC-obj_JobjC@vq_prev)
    jac_with_proj = obj_JobjC @ projector
    inv_jac_with_proj = np.linalg.pinv(jac_with_proj) # pinv(J2@P1)
    vq = vq_prev + inv_jac_with_proj @ new_error

    new_proj = projector - inv_jac_with_proj@jac_with_proj    
    return vq, new_proj



In [73]:
# reset(viz)
q = q0.copy()
viz.display(q0)

DT = 1e-2
goals = [1, 2, 3]
# goals = [1, 2] #OK 
# goals = [1, 3]
for i in range(1000):  # Integrate over 2 second of robot life
    pin.framesForwardKinematics(robot.model, robot.data, q)  # update Forward kinematic

    # TOOL ON THE TABLE Z=[2:3]
    vq, p = get_config_velocity_update_translation(
        q, robot,
        IDX_TOOL, o_Mtable, constraints=(2,3)
    )

    # BASIS TOWARD THE CHECKPOINT ON THE FLOOR XY =[0:2]
    if 2 in goals:
        vq, p = get_config_velocity_update_translation_with_proj(
            q, robot, 
            IDX_BASIS, o_M_floorTarget, constraints=(0,2),
            vq_prev=vq, projector=p
        )

    # GAZE TOWARD THE BALL XYZ =[0:3]
    if 3 in goals:
        vq, p = get_config_velocity_update_translation_with_proj(
            q, robot, 
            IDX_GAZE, o_Mgazegoal, constraints=(0,3),
            vq_prev=vq, projector=p
        )

    q = pin.integrate(robot.model, q, vq*DT)
    viz.display(q)

In [ ]:
# vq, p = get_config_velocity_update_translation(
#     q, robot,
#     IDX_GAZE, o_Mgazegoal, constraints=(0,3)
# )


------------------------

Task 1 ONLY

In [19]:
# reset(viz)
q = q0.copy()
viz.display(q0)

DT = 5e-2
herr = [] # Log the value of the error between tool and goal.
herr2 = [] # Log the value of the error between gaze and ball.
for i in range(100):  # Integrate over 2 second of robot life
    pin.framesForwardKinematics(robot.model, robot.data, q)  # update Forward kinematic
    vq, _ = get_config_velocity_update_translation(q, robot, IDX_TOOL, o_Mtable, constraints=(2,3))
    q = pin.integrate(robot.model, q, vq*DT)
    viz.display(q)